In [1]:
!pip install langchain langchain-classic langchain-community langchain-huggingface langchain-unstructured unstructured unstructured-client "unstructured[html]" chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!unzip chroma_db.zip -d /content/chroma_db

Archive:  chroma_db.zip
   creating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/
  inflating: /content/chroma_db/chroma.sqlite3  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/index_metadata.pickle  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/link_lists.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/header.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/length.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/data_level0.bin  


In [3]:
import requests
from bs4 import BeautifulSoup , Comment
import tempfile
import os
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import Chroma
import re

In [4]:
def load_html(url):
    if not url:
        return ""

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        response = requests.get(
            url,
            headers=headers,
            timeout=10,
            verify=False
        )
        response.raise_for_status()  # raises HTTPError for 4xx/5xx
        return response.text

    except requests.exceptions.RequestException as e:
        print(f"Error loading {url}: {e}")
        raise


In [5]:
def remove_comments(soup):
  for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
    comment.extract()

In [6]:
from google.colab import userdata

embedding_model = HuggingFaceEndpointEmbeddings(
    model = 'sentence-transformers/all-MiniLM-L6-v2',
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

In [48]:
vector_store = Chroma(
  embedding_function=embedding_model,
  persist_directory="chroma_db",
  collection_name="collection_research_guide"
)

In [49]:
def get_structured_docs(docs , url , img_url , faculty_name):

  # Prepare structured document array
  structured_docs = []

  current_title = None
  current_header = None

  # Categories to treat as titles
  TITLE_CATEGORIES = { "title", "pagetitle", "main title" }

  # Categories to treat as headers
  HEADER_CATEGORIES = {"header", "header1", "header2", "header3", "header4", "header5", "header6", "subheading"}

  # Categories to ignore completely
  IGNORE_CATEGORIES = {"figure", "image" }

  for d in docs:
      category = d.metadata.get("category", "").lower()
      text = d.page_content.strip()

      if not text or len(text) <= 5:
          continue

      if category in TITLE_CATEGORIES:
          current_title = text
          current_header = None  # reset header when new title
      elif category in HEADER_CATEGORIES:
          current_header = text
      elif category in IGNORE_CATEGORIES:
          continue  # skip figures/images
      else:
          # Other categories: paragraph, list, table, link, code, quote, etc.
          structured_docs.append(
              Document(
                  page_content=f"faculty_name-{faculty_name} \n Title-{current_title or "Not available"} \n heading-{current_header or "Not available"} \n url-{url or "Not available"} \n img_url-{img_url or "Not Available"} \n\n description - {text}",
                  metadata={
                      "title": current_title or "Not available",
                      "header": current_header or "Not available",
                      "url": url or "Not available",
                  }
              )
          )

  return structured_docs

# load all faculties

In [50]:
html_content = load_html("https://www.iitism.ac.in/all-faculty")

soup = BeautifulSoup(html_content, "lxml")

# remove comments from html
remove_comments(soup)

faculties = []

divs = soup.find("div" , class_ = "faculty-box").find_all("div" , class_ = "event-box-campus")

for div in divs :
  name = div.find("h3").text.strip()
  url = div.find("a")["href"]
  img = div.find("img")["src"]

  faculties.append({
      "name" : name,
      "url" : url,
      "img" : img
  })

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# load each faculy page

In [51]:
for faculty in faculties:

  html_content = load_html(faculty["url"])

  soup = BeautifulSoup(html_content, "lxml")

  # remove comments from html
  remove_comments(soup)

  # filter only the body of the html content
  section = soup.find("section" , class_=["about-imp-sec"])

  # store the html to a tmep file
  with tempfile.NamedTemporaryFile("w", suffix=".html", delete=False, encoding="utf-8") as tmp_file:
    tmp_file.write(str(section))
    tmp_path = tmp_file.name

  print("Temp file path:", tmp_path)

  # load html using Unstructure HTML loader
  loader = UnstructuredHTMLLoader(file_path=tmp_path, mode="elements")
  docs = loader.load()

  # remove the temp file
  os.remove(tmp_path)

  structured_docs = get_structured_docs(docs , faculty["url"] , faculty["img"] , faculty["name"])

  if not structured_docs:
      continue

  vector_store.add_documents(structured_docs)
  vector_store.persist()

  print(f"Added {len(structured_docs)} docs to db of proff {faculty["name"]}")

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmp_uvmzw9v.html


/tmp/ipython-input-1011964166.py:33: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


Added 21 docs to db of proff Prof. A Antony Selvan


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Temp file path: /tmp/tmpcozdlp2j.html
Added 42 docs to db of proff Prof. Vivekananda Mukherjee


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmp5uhq4aso.html
Added 71 docs to db of proff Prof. Y. M. Asha Latha


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmp2b6jyu0k.html
Added 108 docs to db of proff Prof. Zafar Alam


In [47]:
structured_docs[:10]

[Document(metadata={'title': 'Prof.A Antony Selvan', 'header': 'Not available', 'url': 'https://www.iitism.ac.in/faculty-details?faculty=antony'}, page_content='faculty_name-Prof. A Antony Selvan \n Title-Prof.A Antony Selvan \n heading-Not available \n url-https://www.iitism.ac.in/faculty-details?faculty=antony \n img_url-https://www.iitism.ac.in/facultyImages/Prof. A Antony Selvan.jpg \n\n description - Designation: Assistant Professor'),
 Document(metadata={'title': 'Prof.A Antony Selvan', 'header': 'Not available', 'url': 'https://www.iitism.ac.in/faculty-details?faculty=antony'}, page_content='faculty_name-Prof. A Antony Selvan \n Title-Prof.A Antony Selvan \n heading-Not available \n url-https://www.iitism.ac.in/faculty-details?faculty=antony \n img_url-https://www.iitism.ac.in/facultyImages/Prof. A Antony Selvan.jpg \n\n description - Department: Mathematics & Computing'),
 Document(metadata={'title': 'Prof.A Antony Selvan', 'header': 'Not available', 'url': 'https://www.iitism.

In [38]:
c = set()

for d in docs:
  c.add(d.metadata["category"])

  if d.metadata["category"] == "NarrativeText":
    print(d.page_content)

c

About Me: I am currently an Assistant Professor in the Department of Mathematics and Computing at the Indian Institute of Technology (Indian School of Mines), Dhanbad. Prior to this, I completed a postdoctoral fellowship at various esteemed institutions. My research focuses on computational harmonic analysis, with a particular emphasis on mathematical sampling theory and its applications in signal processing.
1. Antony Selvan, A. and Radha, R. Sampling and reconstruction in shift-invariant spaces on Rd, Annali di Matematica Pura ed Applicata, 194 (4). no. 6, 1683-1706, 2015. (Q1, IF-1.0, MCQ-0.95) 2. Antony Selvan, A. and Radha, R, Sampling and reconstruction in shift-invariant spaces of B-spline functions, Acta Applicandae Mathematicae, 145, 175-192, 2016. (Q2, IF-1.4, MCQ-1.04) 3. Antony Selvan, A. and Radha, R. Separation of zeros and a Hermite interpolation based frame algorithm for bandlimited functions, Sampling Theory in Signal and Image Processing, 15, 21-35, 2016. (MCQ-0.78) 4

{'Image', 'ListItem', 'NarrativeText', 'Title', 'UncategorizedText'}

In [56]:
vector_store.similarity_search(
    query = "underground construction",
    k=10
)

[Document(metadata={'source': '/content/1766923003349641_12.csv', 'row': 10}, page_content='Ongoing Projects \n\n \ufeff"Sl. No.": 201\nTitle of the Project: ToAleI-rPaonwt Ceroendt rVoils fioorn  RSedstuecmeds  fHoarr Lmoown-ilcis hatn adn Tdo Lroqwue- vRisipibpilleits a t\nUnderground Mining Environments\nFunding Agency: TEXMiN (Technology Innovation in\nExploration & Mining Foundation),\nName of PI: Prof. Sudhakar Kumawat\nDepartment of PI: Mathematics and Computing'),
 Document(metadata={'title': 'Papers in conference abstract volumes / presented', 'url': 'https://www.iitism.ac.in/faculty-details?faculty=rksinha', 'header': 'Not available'}, page_content='faculty_name-Prof. Rabindra Kumar Sinha \n Title-Papers in conference abstract volumes / presented \n heading-Not available \n url-https://www.iitism.ac.in/faculty-details?faculty=rksinha \n img_url-https://www.iitism.ac.in/storage/FacultyDetails/IMG_173892373367a5ded5736f4.JPG \n\n description - K Sudhakar, RK Sinha, SR Naik (202

In [57]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive("chroma_db", 'zip', "chroma_db")

# Download the zip
files.download("chroma_db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>